生成答案是原始文本的Multiple Choice的prompt（考虑了多种语言的格式，使用4options）

In [4]:
from datasets import load_dataset
import os
import json

# Load the dataset
dataset = load_dataset("fzkuji/cmb", 'CMB-Exam', trust_remote_code=True)

# Define the save path
save_path = f"./data/llama-factory"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    data_to_save = []
    for item in data:
        # Extract the option texts and generate option letters dynamically
        option_texts = []
        option_letters = []
        for idx, option in enumerate(item['option']):
            # Handle different data structures for options
            if isinstance(option, dict):
                option_text = option.get('value', '')
            else:
                option_text = str(option)
            option_texts.append(option_text)
            option_letters.append(chr(65 + idx))  # Generate letters 'A', 'B', etc.
        
        # Create a mapping for the current item
        dict_num = {letter: idx for idx, letter in enumerate(option_letters)}
        
        # Process the answer keys, assuming Answer is a single string of concatenated letters (e.g., "ABC")
        answer_keys = item['answer']

        if isinstance(answer_keys, str):
            # 判断是否为单选题且只有ABCDE五个情况，如果是多选则跳过
            if len(answer_keys) == 1 and answer_keys in ['A', 'B', 'C', 'D', 'E'] and item['question_type'] == '单项选择题':
                answer_keys = [answer_keys]
            else:
                print(f"Warning: Multiple answers detected for item '{item['question']}', skipping...")
                continue
        else:
            # Skip if 'Answer' is not a string or list
            print(f"Warning: Unexpected type for 'Answer' in item '{item['question']}': {type(answer_keys)}")
            continue
        
        # Map the answer keys to the actual option texts, but skip keys that are out of range
        answer_text = []
        for ans in answer_keys:
            if ans in dict_num:
                idx = dict_num[ans]
                answer_text.append(option_texts[idx])
            else:
                print(f"Warning: Answer '{ans}' not found in options for item '{item['question']}'")
        
        # Construct the input text
        input_text = f"问题：{item['question']}\n选项：\n" + "\n".join(
            [f"\t{text}" for text in option_texts]
        )
        
        # Add the instruction, input, and output to the data
        if answer_text:
            instruction = (
                f"假设您是一名医生，正在参加{item['exam_type']}中的{item['exam_class']}考试，科目为{item['exam_subject']}，题型为{item['question_type']}。请回答以下选择题，直接给出答案。"
            )
            data_to_save.append({
                "instruction": instruction,
                "input": input_text,
                "output": "，".join(answer_text)  # Join multiple answers with commas
            })
        else:
            print(f"Warning: No valid answers for item '{item['question']}', skipping...")
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['val'], 'validation.json')
save_as_json(dataset['test'], 'test.json')
